In [1]:
#### url:https://redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile
### webscraping_bus route details in red bus:South_bengal

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd

# Initialize the WebDriver and open the website
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile')

# List to store the collected bus route data
bus_route_data = []

# Function to collect route details from the page
def route_details():
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "route_link")))
        route_elements = driver.find_elements(By.CLASS_NAME, "route")
        
        route_links = [route.get_attribute("href") for route in route_elements]
        route_names = [route.get_attribute("title").strip() for route in route_elements]
        
        # Collect the route details for this page and return them
        return [{'route_name': name, 'route_link': link} for name, link in zip(route_names, route_links)]
        
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Function to navigate through all the pages and scrape route details
def scrape_all_pages():
    all_routes = []
    try:
        # Loop through the first 5 pages (or adjust the range as needed)
        for page in range(1, 6):  # Assuming there are 5 pages
            if page > 1:
                # Wait for the pagination tab of the current page to be clickable
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                # Scroll to the pagination tab and click it
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the new page to load
            
            # Collect the route details after navigating to the page (either the first page or after pagination)
            all_routes.extend(route_details())

    except Exception as e:
        print(f"Error occurred while accessing page {page}: {str(e)}")
    return all_routes

# Function to scrape bus details for each route
def scrape_bus_details(route_name, route_link):
    try:
        driver.get(route_link)
        time.sleep(5)

        # Try to click the "View Buses" button if available
        try:
            view_buses = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses)
            time.sleep(5)
        except:
            print(f"No 'View Buses' button found for {route_link}")

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract data from the page
        bus_names = driver.find_elements(By.CSS_SELECTOR, "div.travels.lh-24.f-bold.d-color")
        bus_types = driver.find_elements(By.CSS_SELECTOR, "div.bus-type.f-12.m-top-16.l-color.evBus")
        departure_times = driver.find_elements(By.CSS_SELECTOR, "div.dp-time.f-19.d-color.f-bold")
        durations = driver.find_elements(By.CSS_SELECTOR, "div.dur.l-color.lh-24")
        reaching_times = driver.find_elements(By.CSS_SELECTOR, "div.bp-time.f-19.d-color.disp-Inline")
        star_ratings = driver.find_elements(By.CSS_SELECTOR, "div.lh-18.rating.rat-red")
        prices = driver.find_elements(By.CSS_SELECTOR, "div.fare.d-block")
        seat_availabilities = driver.find_elements(By.CSS_SELECTOR, "div.seat-left.m-top-16")
    
        bus_route_details = []
    
        # Loop through each bus and store its details
        for i in range(len(bus_names)):
            bus_detail = {
                "State": "South Bengal",
                "Route Name": route_name,
                "Route Link": route_link,
                "Bus Name": bus_names[i].text,
                "Bus Type": bus_types[i].text if i < len(bus_types) else "N/A",
                "Departure Time": departure_times[i].text if i < len(departure_times) else "N/A",
                "Duration": durations[i].text if i < len(durations) else "N/A",
                "Reaching Time": reaching_times[i].text if i < len(reaching_times) else "N/A",
                "Rating": star_ratings[i].text if i < len(star_ratings) else "N/A",
                "Price": prices[i].text if i < len(prices) else "N/A",
                "Seats Available": seat_availabilities[i].text if i < len(seat_availabilities) else "N/A"
            }
            bus_route_details.append(bus_detail)
                     
        return bus_route_details
    except Exception as e:
        print(f"Error occurred while accessing {route_link}: {str(e)}")
        return []   

# Scrape all route details
routes = scrape_all_pages()

# For each route, scrape bus details and add to the bus_route_data list
for route in routes:
    bus_details = scrape_bus_details(route["route_name"], route["route_link"])
    if bus_details:
        bus_route_data.extend(bus_details)  # Add bus details to the bus_route_data list

# Convert bus_route_data to a pandas DataFrame
df = pd.DataFrame(bus_route_data)

# Save the DataFrame to a CSV file
df.to_csv('South_bengal_bus_details.csv', index=False)

# Optionally, print the collected bus details
print("Data saved to South_bengal_bus_details.csv")

# Close the driver after scraping
driver.quit()


No 'View Buses' button found for https://www.redbus.in/bus-tickets/jhargram-to-kolkata
No 'View Buses' button found for https://www.redbus.in/bus-tickets/digha-to-durgapur
No 'View Buses' button found for https://www.redbus.in/bus-tickets/kolkata-to-kolaghat
No 'View Buses' button found for https://www.redbus.in/bus-tickets/kirnahar-west-bengal-to-kolkata
No 'View Buses' button found for https://www.redbus.in/bus-tickets/durgapur-to-barasat-west-bengal
No 'View Buses' button found for https://www.redbus.in/bus-tickets/illambazar-to-kolkata
No 'View Buses' button found for https://www.redbus.in/bus-tickets/purulia-to-durgapur
No 'View Buses' button found for https://www.redbus.in/bus-tickets/durgapur-to-berhampore
No 'View Buses' button found for https://www.redbus.in/bus-tickets/kolkata-to-debra
No 'View Buses' button found for https://www.redbus.in/bus-tickets/berhampore-to-durgapur
No 'View Buses' button found for https://www.redbus.in/bus-tickets/barasat-west-bengal-to-midnapore
No 

In [3]:
df

,State,Route Name,Route Link,Bus Name,Bus Type,Departure Time,Duration,Reaching Time,Rating,Price,Seats Available
0,South Bengal,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-BANKURA - KARUNAMOYEE - A/C - 15:15 - 1500,AC Seater (2+2),17:00,04h 00m,21:00,2.4,INR 300,31 Seats available
1,South Bengal,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KARUNAMOYEE - 16:10 (BEL DEPOT...,Non AC Seater (2+3),17:25,04h 00m,21:25,N/A,INR 155,46 Seats available
2,South Bengal,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KOLKATA - 16:30 (HOWRAH DEPOT)...,Non AC Seater (2+3),17:40,03h 30m,21:10,N/A,INR 150,48 Seats available
3,South Bengal,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KARUNAMOYEE - 16:40 (ASANSOL D...,Non AC Seater (2+3),17:50,04h 10m,22:00,N/A,INR 155,45 Seats available
4,South Bengal,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,Greenline (Karunamoyee),Volvo A/C Seater (2+2),15:39,03h 51m,19:30,N/A,INR 486,12 Seats available
...,...,...,...,...,...,...,...,...,...,...,...
302,South Bengal,Kolkata to Ramnagar (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-r...,Bengal Surface Transport,A/C Seater Push Back (2+3),23:59,04h 46m,04:45,N/A,319,N/A
303,South Bengal,Kolkata to Ashari,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - MIDNAPUR - A/C - 16:30 (MID DE...,AC Seater (2+2),16:30,07h 30m,00:00,N/A,INR 135,N/A
304,South Bengal,Kolkata to Ashari,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - JHARGRAM - VIA - LODHASULI - 1...,Non AC Seater (2+3),16:50,07h 10m,00:00,N/A,INR 82,N/A
305,South Bengal,Kolkata to Ashari,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - MIDNAPUR - 18:00 (MID DEPOT) -...,Non AC Seater (2+3),18:00,06h 00m,00:00,N/A,INR 82,N/A
